##### Bản quyền 2020 The TensorFlow Authors.

In [15]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/text/tutorials/word2vec"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/text/blob/master/docs/tutorials/word2vec.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/text/blob/master/docs/tutorials/word2vec.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/text/docs/tutorials/word2vec.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

# word2vec - Word Embeddings với Skip-gram và Negative Sampling

## Giới thiệu

word2vec không phải là một thuật toán đơn lẻ, mà là một **họ** kiến trúc mô hình và kỹ thuật tối ưu dùng để học **vector biểu diễn từ (word embedding)** từ các tập dữ liệu văn bản lớn. Các embedding học được thông qua word2vec đã chứng minh hiệu quả trong rất nhiều bài toán xử lý ngôn ngữ tự nhiên (NLP) downstream tasks.

**Lưu ý**: Tutorial này dựa trên hai bài báo nền tảng:
- [Efficient estimation of word representations in vector space](https://arxiv.org/pdf/1301.3781.pdf)  
- [Distributed representations of words and phrases and their compositionality](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf)

Đây không phải là implementation chính xác của các paper, mà nhằm minh họa các ý tưởng chính.

### Ý tưởng trực quan của word2vec

Mục tiêu của word2vec là ánh xạ **mỗi từ** trong từ vựng vào một **vector thực nhiều chiều** sao cho:

- Những từ xuất hiện trong **ngữ cảnh giống nhau** sẽ có **vector gần nhau** trong không gian embedding.
- Ví dụ phổ biến:
  - `king - man + woman ≈ queen`
  - `walk` và `walked` nằm gần nhau vì thường xuất hiện trong các câu giống nhau.
  - Các từ như "guitar, drum, bass, piano" thường đi chung trong ngữ cảnh "ban nhạc", nên nằm thành một cụm.

Các vector này được học **thuần túy từ dữ liệu**, không cần gán nhãn thủ công cho từng từ.

### Hai phương pháp học biểu diễn từ

Các paper đề xuất hai phương pháp để học biểu diễn từ:

*   **Continuous bag-of-words model (CBOW)**: dự đoán từ ở giữa dựa trên các từ ngữ cảnh xung quanh. Ngữ cảnh bao gồm một vài từ trước và sau từ hiện tại (từ ở giữa). Kiến trúc này được gọi là mô hình bag-of-words vì thứ tự của các từ trong ngữ cảnh không quan trọng.
    - **Đầu vào**: các từ ngữ cảnh xung quanh.
    - **Đầu ra**: từ trung tâm.
    - Bỏ qua thứ tự bên trong ngữ cảnh (bag-of-words).

*   **Continuous skip-gram model**: dự đoán các từ trong một phạm vi nhất định trước và sau từ hiện tại trong cùng một câu. 
    - **Đầu vào**: từ trung tâm.
    - **Đầu ra**: từng từ ngữ cảnh xung quanh trong một cửa sổ `window_size`.
    - Một câu được chuyển thành rất nhiều cặp **(target_word, context_word)**.

Bạn sẽ sử dụng phương pháp **skip-gram** trong tutorial này. Đầu tiên, bạn sẽ khám phá skip-grams và các khái niệm khác bằng cách sử dụng một câu đơn để minh họa. Tiếp theo, bạn sẽ huấn luyện mô hình word2vec của riêng mình trên một tập dữ liệu nhỏ. Tutorial này cũng chứa code để xuất các embeddings đã huấn luyện và trực quan hóa chúng trong [TensorFlow Embedding Projector](http://projector.tensorflow.org/).

## Skip-gram và Negative Sampling

Trong khi mô hình bag-of-words dự đoán một từ dựa trên ngữ cảnh lân cận, mô hình skip-gram dự đoán ngữ cảnh (hoặc các từ lân cận) của một từ, khi biết chính từ đó. Mô hình được huấn luyện trên các skip-grams, là các n-grams cho phép bỏ qua các tokens (xem sơ đồ bên dưới để có ví dụ). Ngữ cảnh của một từ có thể được biểu diễn thông qua một tập hợp các cặp skip-gram `(target_word, context_word)` trong đó `context_word` xuất hiện trong ngữ cảnh lân cận của `target_word`.

Xét câu sau đây có tám từ:

> The wide road shimmered in the hot sun.

Các từ ngữ cảnh cho mỗi từ trong 8 từ của câu này được xác định bởi kích thước cửa sổ (window size). Kích thước cửa sổ xác định phạm vi các từ ở mỗi bên của `target_word` mà có thể được coi là `context word`. Dưới đây là bảng các skip-grams cho các từ mục tiêu dựa trên các kích thước cửa sổ khác nhau.

**Lưu ý**: Trong tutorial này, kích thước cửa sổ `n` có nghĩa là n từ ở mỗi bên với tổng phạm vi cửa sổ là 2*n+1 từ xung quanh một từ.

![word2vec_skipgrams](https://tensorflow.org/text/tutorials/images/word2vec_skipgram.png)

### Hàm mục tiêu của Skip-gram

Mục tiêu huấn luyện của mô hình skip-gram là tối đa hóa xác suất dự đoán các từ ngữ cảnh khi biết từ mục tiêu. Cho một chuỗi các từ *w<sub>1</sub>, w<sub>2</sub>, ... w<sub>T</sub>*, mục tiêu có thể được viết dưới dạng xác suất log trung bình:

Cho dãy từ \(w_1, w_2, \dots, w_T\). Với mỗi vị trí \(t\), xét một cửa sổ ngữ cảnh kích thước \(c\) (hai bên).

Hàm mục tiêu là **tối đa hóa log-xác suất trung bình**:

$$
\frac{1}{T} \sum_{t=1}^{T} \sum_{-c \le j \le c, j \neq 0} \log P(w_{t+j} \mid w_t)
$$

![word2vec_skipgram_objective](https://tensorflow.org/text/tutorials/images/word2vec_skipgram_objective.png)

trong đó `c` là kích thước của ngữ cảnh huấn luyện. Công thức skip-gram cơ bản định nghĩa xác suất này bằng cách sử dụng hàm softmax.

Trong mô hình skip-gram cơ bản, xác suất sử dụng **softmax toàn vocab**:

$$
P(w_O \mid w_I) = \frac{\exp(v'_{w_O} \cdot v_{w_I})}{\sum_{w=1}^{W} \exp(v'_w \cdot v_{w_I})}
$$

![word2vec_full_softmax](https://tensorflow.org/text/tutorials/images/word2vec_full_softmax.png)

trong đó:
- *v* và *v'* là các biểu diễn vector mục tiêu (target) và ngữ cảnh (context) của các từ
- *W* là kích thước từ vựng
- *v<sub>w<sub>I</sub></sub>*: vector embedding của từ khi làm **target**
- *v'<sub>w<sub>O</sub></sub>*: vector embedding của từ khi làm **context**

### Vấn đề với Full Softmax

Việc tính toán mẫu số của công thức này liên quan đến việc thực hiện softmax đầy đủ trên toàn bộ các từ trong từ vựng, thường rất lớn (10<sup>5</sup>-10<sup>7</sup>) từ. Điều này rất tốn kém về mặt tính toán và chậm, không phù hợp cho huấn luyện trên tập dữ liệu lớn.

### Noise Contrastive Estimation (NCE) và Negative Sampling

Hàm loss [noise contrastive estimation](https://www.tensorflow.org/api_docs/python/tf/nn/nce_loss) (NCE) là một phép xấp xỉ hiệu quả cho softmax đầy đủ. Với mục tiêu là học word embeddings thay vì mô hình hóa phân phối từ, loss NCE có thể được [đơn giản hóa](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) để sử dụng negative sampling.

Để tránh softmax toàn vocab, paper sử dụng **Noise Contrastive Estimation (NCE)**, có thể đơn giản thành **negative sampling**.

### Ý tưởng trực quan của Negative Sampling

Mục tiêu negative sampling đơn giản hóa cho một từ mục tiêu là phân biệt từ ngữ cảnh với `num_ns` mẫu negative được lấy từ phân phối nhiễu *P<sub>n</sub>(w)* của các từ. Cụ thể hơn, một phép xấp xỉ hiệu quả của softmax đầy đủ trên từ vựng là, đối với một cặp skip-gram, đặt loss cho một từ mục tiêu như một bài toán phân loại giữa từ ngữ cảnh và `num_ns` mẫu negative.

Thay vì trực tiếp mô hình hóa phân phối \(P(w_O \mid w_I)\), ta biến bài toán thành **phân loại nhị phân**:

- Với **mỗi cặp đúng** (positive) \((t, c)\) mà \(c\) thực sự nằm trong ngữ cảnh của \(t\),
  - ta sinh thêm \(k\) cặp **sai** (negative) \((t, n_i)\), trong đó \(n_i\) là các từ **không** xuất hiện trong cửa sổ ngữ cảnh của \(t\).
- Mô hình cần học để:
  - **Positive**: \( \sigma(v'_c \cdot v_t) \approx 1 \)
  - **Negative**: \( \sigma(v'_{n_i} \cdot v_t) \approx 0 \)

Ở đây \( \sigma \) là hàm sigmoid.

### Hàm Loss Negative Sampling

Với 1 cặp positive \((t, c)\) và \(k\) negative \((t, n_1), \dots, (t, n_k)\), hàm loss là:

$$
L = - \Big[ \log \sigma(v'_c \cdot v_t) + \sum_{i=1}^{k} \log \sigma(- v'_{n_i} \cdot v_t) \Big]
$$

- Thành phần thứ nhất khuyến khích dot-product cho cặp đúng **cao** (sigmoid gần 1).
- Thành phần thứ hai khuyến khích dot-product cho cặp sai **thấp** (sigmoid gần 0).

Một mẫu negative được định nghĩa là một cặp `(target_word, context_word)` sao cho `context_word` không xuất hiện trong vùng lân cận `window_size` của `target_word`. Đối với câu ví dụ, đây là một vài mẫu negative tiềm năng (khi `window_size` là `2`):

```
(hot, shimmered)
(wide, hot)
(wide, sun)
```

Trong phần tiếp theo, bạn sẽ tạo skip-grams và negative samples cho một câu đơn. Bạn cũng sẽ học về các kỹ thuật subsampling và huấn luyện một mô hình phân loại cho các ví dụ huấn luyện positive và negative sau này trong tutorial.

## Cài Đặt và Import Thư Viện

In [16]:
%pip install tensorflow
import io
import re
import string
import tqdm

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

   ---------------------------------------- 0.0/331.9 MB ? eta -:--:--
   ---------------------------------------- 0.8/331.9 MB 8.5 MB/s eta 0:00:40
   - -------------------------------------- 8.4/331.9 MB 27.4 MB/s eta 0:00:12
   -- ------------------------------------- 17.3/331.9 MB 33.1 MB/s eta 0:00:10
   --- ------------------------------------ 28.0/331.9 MB 38.7 MB/s eta 0:00:08
   ---- ----------------------------------- 37.0/331.9 MB 39.2 MB/s eta 0:00:08
   ----- ---------------------------------- 46.9/331.9 MB 42.1 MB/s eta 0:00:07
   ------ --------------------------------- 55.6/331.9 MB 41.2 MB/s eta 0:00:07
   -------- ------------------------------- 66.8/331.9 MB 42.6 MB/s eta 0:00:07
   --------- ------------------------------ 77.3/331.9 MB 43.7 MB/s eta 0:00:06
   ---------- ----------------------------- 87.3/331.9 MB 44.2 MB/s eta 0:00:06
   ----------- ---------------------------- 98.0/331.9 MB 44.7 MB/s eta 0:00:06
   ------------ -------------------------- 109.6/331

In [17]:
# Load the TensorBoard notebook extension
%load_ext tensorboard

In [18]:
SEED = 42
AUTOTUNE = tf.data.AUTOTUNE

### Vector Hóa Một Câu Ví Dụ

Xét câu sau đây:

> The wide road shimmered in the hot sun.

**Tokenize câu** (tách thành các từ riêng lẻ):

In [19]:
sentence = "The wide road shimmered in the hot sun"
tokens = list(sentence.lower().split())
print(len(tokens))

8


Tạo một **từ vựng (vocabulary)** để lưu ánh xạ từ tokens sang chỉ số nguyên:

In [20]:
vocab, index = {}, 1  # start indexing from 1
vocab['<pad>'] = 0  # add a padding token
for token in tokens:
  if token not in vocab:
    vocab[token] = index
    index += 1
vocab_size = len(vocab)
print(vocab)

{'<pad>': 0, 'the': 1, 'wide': 2, 'road': 3, 'shimmered': 4, 'in': 5, 'hot': 6, 'sun': 7}


Tạo một **từ vựng nghịch đảo (inverse vocabulary)** để lưu ánh xạ từ chỉ số nguyên sang tokens:

In [21]:
inverse_vocab = {index: token for token, index in vocab.items()}
print(inverse_vocab)

{0: '<pad>', 1: 'the', 2: 'wide', 3: 'road', 4: 'shimmered', 5: 'in', 6: 'hot', 7: 'sun'}


**Vector hóa câu** của bạn (chuyển thành dãy số nguyên):

In [22]:
example_sequence = [vocab[word] for word in tokens]
print(example_sequence)

[1, 2, 3, 4, 5, 1, 6, 7]


### Tạo Skip-grams Từ Một Câu

Module `tf.keras.preprocessing.sequence` cung cấp các hàm hữu ích giúp đơn giản hóa việc chuẩn bị dữ liệu cho word2vec. Bạn có thể sử dụng `tf.keras.preprocessing.sequence.skipgrams` để tạo các cặp skip-gram từ `example_sequence` với một `window_size` cho trước từ các tokens trong phạm vi `[0, vocab_size)`.

**Lưu ý**: `negative_samples` được đặt thành `0` ở đây, vì việc batching các negative samples được tạo bởi hàm này yêu cầu một chút code. Bạn sẽ sử dụng một hàm khác để thực hiện negative sampling trong phần tiếp theo.

In [23]:
window_size = 2
positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
      example_sequence,
      vocabulary_size=vocab_size,
      window_size=window_size,
      negative_samples=0,
      seed=SEED)
print(len(positive_skip_grams))

26


In ra một vài **positive skip-grams** (các cặp skip-gram dương):

In [24]:
for target, context in positive_skip_grams[:5]:
  print(f"({target}, {context}): ({inverse_vocab[target]}, {inverse_vocab[context]})")

(5, 6): (in, hot)
(4, 1): (shimmered, the)
(4, 2): (shimmered, wide)
(7, 6): (sun, hot)
(1, 6): (the, hot)


### Negative Sampling Cho Một Skip-gram

Hàm `skipgrams` trả về tất cả các cặp skip-gram positive bằng cách trượt qua một phạm vi cửa sổ cho trước. Để tạo thêm các cặp skip-gram sẽ phục vụ như negative samples cho huấn luyện, bạn cần lấy mẫu các từ ngẫu nhiên từ từ vựng. Sử dụng hàm `tf.random.log_uniform_candidate_sampler` để lấy mẫu `num_ns` số lượng negative samples cho một từ mục tiêu cho trước trong một cửa sổ. Bạn có thể gọi hàm trên từ mục tiêu của một skip-gram và truyền từ ngữ cảnh làm lớp thực (true class) để loại trừ nó khỏi việc được lấy mẫu.

**Điểm quan trọng**: `num_ns` (số lượng negative samples cho mỗi từ ngữ cảnh positive) trong khoảng `[5, 20]` được [chứng minh là hoạt động](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) tốt nhất cho các tập dữ liệu nhỏ hơn, trong khi `num_ns` trong khoảng `[2, 5]` là đủ cho các tập dữ liệu lớn hơn.

In [25]:
# Get target and context words for one positive skip-gram.
target_word, context_word = positive_skip_grams[0]

# Set the number of negative samples per positive context.
num_ns = 4

context_class = tf.reshape(tf.constant(context_word, dtype="int64"), (1, 1))
negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
    true_classes=context_class,  # class that should be sampled as 'positive'
    num_true=1,  # each positive skip-gram has 1 positive context class
    num_sampled=num_ns,  # number of negative context words to sample
    unique=True,  # all the negative samples should be unique
    range_max=vocab_size,  # pick index of the samples from [0, vocab_size]
    seed=SEED,  # seed for reproducibility
    name="negative_sampling"  # name of this operation
)
print(negative_sampling_candidates)
print([inverse_vocab[index.numpy()] for index in negative_sampling_candidates])

tf.Tensor([2 1 4 3], shape=(4,), dtype=int64)
['wide', 'the', 'shimmered', 'road']


### Xây Dựng Một Ví Dụ Huấn Luyện

Đối với một skip-gram positive `(target_word, context_word)` cho trước, bây giờ bạn cũng có `num_ns` từ ngữ cảnh negative được lấy mẫu mà không xuất hiện trong vùng lân cận kích thước cửa sổ của `target_word`. Gộp `1` từ `context_word` positive và `num_ns` từ ngữ cảnh negative vào một tensor. Điều này tạo ra một tập hợp các skip-grams positive (được gán nhãn là `1`) và các negative samples (được gán nhãn là `0`) cho mỗi từ mục tiêu.

In [26]:
# Reduce a dimension so you can use concatenation (in the next step).
squeezed_context_class = tf.squeeze(context_class, 1)

# Concatenate a positive context word with negative sampled words.
context = tf.concat([squeezed_context_class, negative_sampling_candidates], 0)

# Label the first context word as `1` (positive) followed by `num_ns` `0`s (negative).
label = tf.constant([1] + [0]*num_ns, dtype="int64")
target = target_word


Kiểm tra ngữ cảnh và các nhãn tương ứng cho từ mục tiêu từ ví dụ skip-gram ở trên:

In [27]:
print(f"target_index    : {target}")
print(f"target_word     : {inverse_vocab[target_word]}")
print(f"context_indices : {context}")
print(f"context_words   : {[inverse_vocab[c.numpy()] for c in context]}")
print(f"label           : {label}")

target_index    : 5
target_word     : in
context_indices : [6 2 1 4 3]
context_words   : ['hot', 'wide', 'the', 'shimmered', 'road']
label           : [1 0 0 0 0]


Một tuple `(target, context, label)` của các tensors tạo thành một ví dụ huấn luyện cho việc huấn luyện mô hình word2vec skip-gram negative sampling của bạn. Lưu ý rằng target có shape `(1,)` trong khi context và label có shape `(1+num_ns,)`

In [28]:
print("target  :", target)
print("context :", context)
print("label   :", label)

target  : 5
context : tf.Tensor([6 2 1 4 3], shape=(5,), dtype=int64)
label   : tf.Tensor([1 0 0 0 0], shape=(5,), dtype=int64)


### Tóm Tắt

Sơ đồ này tóm tắt quy trình tạo một ví dụ huấn luyện từ một câu:

![word2vec_negative_sampling](https://tensorflow.org/text/tutorials/images/word2vec_negative_sampling.png)

Lưu ý rằng các từ `temperature` và `code` không phải là một phần của câu đầu vào. Chúng thuộc về từ vựng giống như một số chỉ số khác được sử dụng trong sơ đồ trên.

## Tổng Hợp Tất Cả Các Bước Vào Một Hàm

### Bảng Lấy Mẫu Skip-gram (Skip-gram Sampling Table)

### Subsampling các từ xuất hiện quá thường

Một tập dữ liệu lớn có nghĩa là từ vựng lớn hơn với số lượng cao hơn các từ xuất hiện thường xuyên hơn như stopwords. Các ví dụ huấn luyện thu được từ việc lấy mẫu các từ xuất hiện phổ biến (như `the`, `is`, `on`) không thêm nhiều thông tin hữu ích cho mô hình để học. [Mikolov et al.](https://papers.nips.cc/paper/5021-distributed-representations-of-words-and-phrases-and-their-compositionality.pdf) đề xuất subsampling các từ thường xuyên như một phương pháp hữu ích để cải thiện chất lượng embedding.

Trong tập dữ liệu lớn, các từ như `"the"`, `"is"`, `"and"` xuất hiện rất nhiều nhưng ít đóng góp thông tin.

Paper của Mikolov đề xuất **subsampling** để:
- Giảm số lượng ví dụ không hữu ích.
- Cải thiện chất lượng embedding.

Hàm `tf.keras.preprocessing.sequence.skipgrams` chấp nhận một đối số sampling table để mã hóa xác suất lấy mẫu bất kỳ token nào. Bạn có thể sử dụng `tf.keras.preprocessing.sequence.make_sampling_table` để tạo một bảng lấy mẫu xác suất dựa trên thứ hạng tần suất từ và truyền nó cho hàm `skipgrams`. Kiểm tra các xác suất lấy mẫu cho `vocab_size` là 10.

Trong notebook:
- Hàm `make_sampling_table(vocab_size)` tạo một bảng xác suất dựa trên phân bố kiểu **Zipf**.
- Bảng này được truyền cho `skipgrams` để quyết định có bỏ qua bớt các từ tần suất cao hay không.

In [29]:
sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(size=10)
print(sampling_table)

[0.00315225 0.00315225 0.00547597 0.00741556 0.00912817 0.01068435
 0.01212381 0.01347162 0.01474487 0.0159558 ]


`sampling_table[i]` biểu thị xác suất lấy mẫu từ thường gặp thứ i trong một tập dữ liệu. Hàm giả định một [phân phối Zipf](https://en.wikipedia.org/wiki/Zipf%27s_law) của tần suất từ để lấy mẫu.

**Điểm quan trọng**: `tf.random.log_uniform_candidate_sampler` đã giả định rằng tần suất từ vựng tuân theo phân phối log-uniform (Zipf). Việc sử dụng phân phối có trọng số này cũng giúp xấp xỉ loss Noise Contrastive Estimation (NCE) với các hàm loss đơn giản hơn cho việc huấn luyện một mục tiêu negative sampling.

Ngoài ra, `tf.random.log_uniform_candidate_sampler` cũng giả định tần suất từ tuân theo phân bố log-uniform (gần với Zipf), giúp việc negative sampling hiệu quả hơn.

### Tạo Dữ Liệu Huấn Luyện

Tổng hợp tất cả các bước được mô tả ở trên vào một hàm có thể được gọi trên một danh sách các câu đã vector hóa thu được từ bất kỳ tập dữ liệu văn bản nào. Lưu ý rằng sampling table được xây dựng trước khi lấy mẫu các cặp từ skip-gram. Bạn sẽ sử dụng hàm này trong các phần sau.

### Hàm `generate_training_data`

Hàm này gom toàn bộ các bước ở trên cho **một tập các câu**:

**Input**:
- `sequences`: list các câu (mỗi câu là một mảng chỉ số).
- `window_size`: kích thước cửa sổ ngữ cảnh.
- `num_ns`: số negative samples cho mỗi positive.
- `vocab_size`, `seed`.

**Bên trong**:
- Tạo `sampling_table` bằng `make_sampling_table`.
- Với mỗi `sequence`:
  - Gọi `skipgrams` để sinh các cặp positive `(target, context)`.
  - Với mỗi cặp:
    - Gọi `log_uniform_candidate_sampler` để sinh negatives.
    - Tạo `context = [context_pos, neg_1, ..., neg_k]`.
    - Tạo `label = [1, 0, ..., 0]`.
    - Append vào các list `targets`, `contexts`, `labels`.

**Output**:
- `targets`, `contexts`, `labels` (Numpy arrays) có cùng số phần tử – tổng số mẫu huấn luyện.

In [30]:
# Generates skip-gram pairs with negative sampling for a list of sequences
# (int-encoded sentences) based on window size, number of negative samples
# and vocabulary size.
def generate_training_data(sequences, window_size, num_ns, vocab_size, seed):
  # Elements of each training example are appended to these lists.
  targets, contexts, labels = [], [], []

  # Build the sampling table for `vocab_size` tokens.
  sampling_table = tf.keras.preprocessing.sequence.make_sampling_table(vocab_size)

  # Iterate over all sequences (sentences) in the dataset.
  for sequence in tqdm.tqdm(sequences):

    # Generate positive skip-gram pairs for a sequence (sentence).
    positive_skip_grams, _ = tf.keras.preprocessing.sequence.skipgrams(
          sequence,
          vocabulary_size=vocab_size,
          sampling_table=sampling_table,
          window_size=window_size,
          negative_samples=0,
          seed=seed)

    # Iterate over each positive skip-gram pair to produce training examples
    # with a positive context word and negative samples.
    for target_word, context_word in positive_skip_grams:
      context_class = tf.expand_dims(
          tf.constant([context_word], dtype="int64"), 1)
      negative_sampling_candidates, _, _ = tf.random.log_uniform_candidate_sampler(
          true_classes=context_class,
          num_true=1,
          num_sampled=num_ns,
          unique=True,
          range_max=vocab_size,
          seed=seed,
          name="negative_sampling")

      # Build context and label vectors (for one target word)
      context = tf.concat([tf.squeeze(context_class,1), negative_sampling_candidates], 0)
      label = tf.constant([1] + [0]*num_ns, dtype="int64")

      # Append each element from the training example to global lists.
      targets.append(target_word)
      contexts.append(context)
      labels.append(label)

  return targets, contexts, labels

## Chuẩn Bị Dữ Liệu Huấn Luyện Cho Word2vec

Với sự hiểu biết về cách làm việc với một câu cho mô hình word2vec dựa trên skip-gram negative sampling, bạn có thể tiến hành tạo các ví dụ huấn luyện từ một danh sách lớn hơn các câu!

### Tải Xuống Text Corpus

Bạn sẽ sử dụng một file văn bản các tác phẩm của Shakespeare cho tutorial này. Thay đổi dòng sau để chạy code này trên dữ liệu của riêng bạn.

Notebook sử dụng dữ liệu Shakespeare:

In [31]:
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')

1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
1115394/1115394 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Đọc văn bản từ file và in ra một vài dòng đầu tiên:

In [32]:
with open(path_to_file) as f:
  lines = f.read().splitlines()
for line in lines[:20]:
  print(line)

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.

All:
We know't, we know't.

First Citizen:
Let us kill him, and we'll have corn at our own price.


Sử dụng các dòng không rỗng để xây dựng một đối tượng `tf.data.TextLineDataset` cho các bước tiếp theo:

Tạo `TextLineDataset` từ file, lọc bỏ dòng rỗng:

In [33]:
text_ds = tf.data.TextLineDataset(path_to_file).filter(lambda x: tf.cast(tf.strings.length(x), bool))

### Vector Hóa Các Câu Từ Corpus

Bạn có thể sử dụng lớp `TextVectorization` để vector hóa các câu từ corpus. Tìm hiểu thêm về việc sử dụng lớp này trong tutorial [Text classification](https://www.tensorflow.org/tutorials/keras/text_classification). Lưu ý từ một vài câu đầu tiên ở trên rằng văn bản cần được chuyển về một dạng chữ và dấu câu cần được loại bỏ. Để làm điều này, định nghĩa một `custom_standardization function` có thể được sử dụng trong lớp TextVectorization.

Dùng lớp `TextVectorization` để:
- Chuẩn hóa (lowercase + bỏ dấu câu).
- Tạo từ vựng (`get_vocabulary()`).
- Biến mỗi dòng thành một sequence chỉ số cố định độ dài `sequence_length`.

In [34]:
# Now, create a custom standardization function to lowercase the text and
# remove punctuation.
def custom_standardization(input_data):
  lowercase = tf.strings.lower(input_data)
  return tf.strings.regex_replace(lowercase,
                                  '[%s]' % re.escape(string.punctuation), '')


# Define the vocabulary size and the number of words in a sequence.
vocab_size = 4096
sequence_length = 10

# Use the `TextVectorization` layer to normalize, split, and map strings to
# integers. Set the `output_sequence_length` length to pad all samples to the
# same length.
vectorize_layer = layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=vocab_size,
    output_mode='int',
    output_sequence_length=sequence_length)

Gọi `TextVectorization.adapt` trên tập dữ liệu văn bản để tạo từ vựng:

In [35]:
vectorize_layer.adapt(text_ds.batch(1024))

Sau khi trạng thái của lớp đã được điều chỉnh để biểu diễn corpus văn bản, từ vựng có thể được truy cập bằng `TextVectorization.get_vocabulary`. Hàm này trả về một danh sách tất cả các tokens từ vựng được sắp xếp (giảm dần) theo tần suất của chúng.

In [36]:
# Save the created vocabulary for reference.
inverse_vocab = vectorize_layer.get_vocabulary()
print(inverse_vocab[:20])

['', '[UNK]', np.str_('the'), np.str_('and'), np.str_('to'), np.str_('i'), np.str_('of'), np.str_('you'), np.str_('my'), np.str_('a'), np.str_('that'), np.str_('in'), np.str_('is'), np.str_('not'), np.str_('for'), np.str_('with'), np.str_('me'), np.str_('it'), np.str_('be'), np.str_('your')]


`vectorize_layer` bây giờ có thể được sử dụng để tạo vectors cho mỗi phần tử trong `text_ds` (một `tf.data.Dataset`). Áp dụng `Dataset.batch`, `Dataset.prefetch`, `Dataset.map`, và `Dataset.unbatch`.

Sau đó:
- Dùng `.batch`, `.prefetch`, `.map(vectorize_layer)`, `.unbatch()` để có dataset các câu dạng index.

In [37]:
# Vectorize the data in text_ds.
text_vector_ds = text_ds.batch(1024).prefetch(AUTOTUNE).map(vectorize_layer).unbatch()

### Lấy Các Sequences Từ Dataset

Bây giờ bạn có một `tf.data.Dataset` của các câu được mã hóa số nguyên. Để chuẩn bị dataset cho việc huấn luyện một mô hình word2vec, làm phẳng (flatten) dataset thành một danh sách các sequences vector câu. Bước này là cần thiết vì bạn sẽ lặp qua mỗi câu trong dataset để tạo ra các ví dụ positive và negative.

**Lưu ý**: Vì hàm `generate_training_data()` được định nghĩa trước đó sử dụng các hàm Python/NumPy không phải TensorFlow, bạn cũng có thể sử dụng `tf.py_function` hoặc `tf.numpy_function` với `tf.data.Dataset.map`.

Chuyển thành list `sequences = list(text_vector_ds.as_numpy_iterator())`.

In [38]:
sequences = list(text_vector_ds.as_numpy_iterator())
print(len(sequences))

32777


Kiểm tra một vài ví dụ từ `sequences`:

In [39]:
for seq in sequences[:5]:
  print(f"{seq} => {[inverse_vocab[i] for i in seq]}")

[ 89 270   0   0   0   0   0   0   0   0] => [np.str_('first'), np.str_('citizen'), '', '', '', '', '', '', '', '']
[138  36 982 144 673 125  16 106   0   0] => [np.str_('before'), np.str_('we'), np.str_('proceed'), np.str_('any'), np.str_('further'), np.str_('hear'), np.str_('me'), np.str_('speak'), '', '']
[34  0  0  0  0  0  0  0  0  0] => [np.str_('all'), '', '', '', '', '', '', '', '', '']
[106 106   0   0   0   0   0   0   0   0] => [np.str_('speak'), np.str_('speak'), '', '', '', '', '', '', '', '']
[ 89 270   0   0   0   0   0   0   0   0] => [np.str_('first'), np.str_('citizen'), '', '', '', '', '', '', '', '']


### Tạo Các Ví Dụ Huấn Luyện Từ Sequences

`sequences` bây giờ là một danh sách các câu được mã hóa số nguyên. Chỉ cần gọi hàm `generate_training_data` được định nghĩa trước đó để tạo các ví dụ huấn luyện cho mô hình word2vec. Để nhắc lại, hàm lặp qua mỗi từ từ mỗi sequence để thu thập các từ ngữ cảnh positive và negative. Độ dài của target, contexts và labels phải giống nhau, biểu thị tổng số ví dụ huấn luyện.

Cuối cùng:
- Gọi `generate_training_data(...)` để thu được `targets`, `contexts`, `labels` cho toàn bộ corpus.

In [ ]:
targets, contexts, labels = generate_training_data(
    sequences=sequences,
    window_size=2,
    num_ns=4,
    vocab_size=vocab_size,
    seed=SEED)

targets = np.array(targets)
contexts = np.array(contexts)
labels = np.array(labels)

print('\n')
print(f"targets.shape: {targets.shape}")
print(f"contexts.shape: {contexts.shape}")
print(f"labels.shape: {labels.shape}")


  2%|▏         | 535/32777 [00:00<00:18, 1779.20it/s]

### Cấu Hình Dataset Để Tối Ưu Hiệu Năng

Để thực hiện batching hiệu quả cho số lượng lớn các ví dụ huấn luyện tiềm năng, sử dụng API `tf.data.Dataset`. Sau bước này, bạn sẽ có một đối tượng `tf.data.Dataset` của các phần tử `(target_word, context_word), (label)` để huấn luyện mô hình word2vec của bạn!

### Xây dựng `tf.data.Dataset` để huấn luyện

Từ các mảng Numpy:
- Tạo dataset: `dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))`
- Shuffle và batch với `BUFFER_SIZE` và `BATCH_SIZE`

Kết quả là một `tf.data.Dataset` có phần tử dạng:
- **Input**: `(target_batch, context_batch)`
- **Label**: `label_batch`

Trong đó:
- `target_batch.shape` ≈ `(batch_size,)`
- `context_batch.shape` ≈ `(batch_size, 1 + num_ns)`
- `label_batch.shape` giống `context_batch.shape`

In [ ]:
BATCH_SIZE = 1024
BUFFER_SIZE = 10000
dataset = tf.data.Dataset.from_tensor_slices(((targets, contexts), labels))
dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset)

<_BatchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


Áp dụng `Dataset.cache` và `Dataset.prefetch` để cải thiện hiệu năng:

In [ ]:
dataset = dataset.cache().prefetch(buffer_size=AUTOTUNE)
print(dataset)

<_PrefetchDataset element_spec=((TensorSpec(shape=(1024,), dtype=tf.int64, name=None), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None)), TensorSpec(shape=(1024, 5), dtype=tf.int64, name=None))>


## Mô Hình và Huấn Luyện

Mô hình word2vec có thể được triển khai như một bộ phân loại để phân biệt giữa các từ ngữ cảnh thực từ skip-grams và các từ ngữ cảnh sai được thu thập thông qua negative sampling. Bạn có thể thực hiện phép nhân tích vô hướng (dot product) giữa các embeddings của từ mục tiêu và từ ngữ cảnh để thu được các dự đoán cho nhãn và tính toán hàm loss so với các nhãn thực trong dataset.

### Mô Hình Word2vec Sử Dụng Keras Subclassing

Sử dụng [Keras Subclassing API](https://www.tensorflow.org/guide/keras/custom_layers_and_models) để định nghĩa mô hình word2vec của bạn với các lớp sau:

* `target_embedding`: Một lớp `tf.keras.layers.Embedding`, tra cứu embedding của một từ khi nó xuất hiện như một từ mục tiêu. Số lượng tham số trong lớp này là `(vocab_size * embedding_dim)`.
* `context_embedding`: Một lớp `tf.keras.layers.Embedding` khác, tra cứu embedding của một từ khi nó xuất hiện như một từ ngữ cảnh. Số lượng tham số trong lớp này giống như trong `target_embedding`, tức là `(vocab_size * embedding_dim)`.
* `dots`: Một lớp `tf.keras.layers.Dot` tính tích vô hướng (dot product) của embeddings mục tiêu và ngữ cảnh từ một cặp huấn luyện.
* `flatten`: Một lớp `tf.keras.layers.Flatten` để làm phẳng kết quả của lớp `dots` thành logits.

Với mô hình subclassed, bạn có thể định nghĩa hàm `call()` chấp nhận các cặp `(target, context)` sau đó có thể được truyền vào lớp embedding tương ứng của chúng. Reshape `context_embedding` để thực hiện tích vô hướng với `target_embedding` và trả về kết quả đã làm phẳng.

### Kiến trúc Mô hình Word2Vec (Keras Subclassing)

Notebook định nghĩa lớp với các thành phần:

**Giải thích**:
- `target_embedding` và `context_embedding` là hai bảng embedding riêng cho **target** và **context**.
- `call` nhận `(target, context)`:
  - Embed target → vector `word_emb`.
  - Embed context → ma trận `context_emb`.
  - Dùng `tf.einsum` để tính dot-product giữa `word_emb` với từng context trong batch → nhận `dots` (logits).

**Điểm quan trọng**: Các lớp `target_embedding` và `context_embedding` cũng có thể được chia sẻ. Bạn cũng có thể sử dụng phép nối (concatenation) của cả hai embeddings làm embedding word2vec cuối cùng.

> **Ghi chú**: Có thể dùng **chung** 1 bảng embedding cho target và context, hoặc kết hợp hai vector để tạo embedding cuối cùng (một số biến thể làm vậy).

In [ ]:
class Word2Vec(tf.keras.Model):
  def __init__(self, vocab_size, embedding_dim):
    super(Word2Vec, self).__init__()
    self.target_embedding = layers.Embedding(vocab_size,
                                      embedding_dim,
                                      name="w2v_embedding")
    self.context_embedding = layers.Embedding(vocab_size,
                                       embedding_dim)

  def call(self, pair):
    target, context = pair
    # target: (batch, dummy?)  # The dummy axis doesn't exist in TF2.7+
    # context: (batch, context)
    if len(target.shape) == 2:
      target = tf.squeeze(target, axis=1)
    # target: (batch,)
    word_emb = self.target_embedding(target)
    # word_emb: (batch, embed)
    context_emb = self.context_embedding(context)
    # context_emb: (batch, context, embed)
    dots = tf.einsum('be,bce->bc', word_emb, context_emb)
    # dots: (batch, context)
    return dots

### Định Nghĩa Hàm Loss và Compile Model

### Hàm Loss và Huấn Luyện

Để đơn giản, bạn có thể sử dụng `tf.keras.losses.CategoricalCrossEntropy` như một thay thế cho loss negative sampling. Nếu bạn muốn viết hàm loss tùy chỉnh của riêng mình, bạn cũng có thể làm như sau:

```python
def custom_loss(x_logit, y_true):
      return tf.nn.sigmoid_cross_entropy_with_logits(logits=x_logit, labels=y_true)
```

Đã đến lúc xây dựng mô hình của bạn! Khởi tạo lớp word2vec của bạn với embedding dimension là 128 (bạn có thể thử nghiệm với các giá trị khác). Compile mô hình với optimizer `tf.keras.optimizers.Adam`.

Trong notebook, để đơn giản:
- Dùng `tf.keras.losses.CategoricalCrossentropy(from_logits=True)`.
  - Mỗi hàng `dots` tương ứng với một target và `1 + num_ns` context.
  - `labels` là one-hot `[1, 0, ..., 0]` (context đầu tiên là positive).

Mô hình được compile với:
- Optimizer: `Adam`
- Loss: `CategoricalCrossentropy(from_logits=True)`
- Metric: `accuracy` (tỷ lệ dự đoán đúng context dương).

In [ ]:
embedding_dim = 128
word2vec = Word2Vec(vocab_size, embedding_dim)
word2vec.compile(optimizer='adam',
                 loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),
                 metrics=['accuracy'])

Cũng định nghĩa một callback để ghi log các thống kê huấn luyện cho TensorBoard:

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir="logs")

Huấn luyện mô hình trên `dataset` cho một số epoch nhất định:

In [ ]:
word2vec.fit(dataset, epochs=20, callbacks=[tensorboard_callback])

Epoch 1/20


I0000 00:00:1721388042.283243   10107 service.cc:146] XLA service 0x7f343003f620 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1721388042.283295   10107 service.cc:154]   StreamExecutor device (0): Tesla T4, Compute Capability 7.5
I0000 00:00:1721388042.283299   10107 service.cc:154]   StreamExecutor device (1): Tesla T4, Compute Capability 7.5
I0000 00:00:1721388042.283302   10107 service.cc:154]   StreamExecutor device (2): Tesla T4, Compute Capability 7.5
I0000 00:00:1721388042.283305   10107 service.cc:154]   StreamExecutor device (3): Tesla T4, Compute Capability 7.5



 1/63 ━━━━━━━━━━━━━━━━━━━━ 1:24 1s/step - accuracy: 0.1924 - loss: 1.6095


16/63 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - accuracy: 0.2070 - loss: 1.6093 


33/63 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.2126 - loss: 1.6091

...

Epoch 20/20



 1/63 ━━━━━━━━━━━━━━━━━━━━ 0s 6ms/step - accuracy: 0.8857 - loss: 0.4731


25/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8870 - loss: 0.4757


49/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8876 - loss: 0.4757


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8878 - loss: 0.4754


TensorBoard bây giờ hiển thị độ chính xác (accuracy) và loss của mô hình word2vec:

TensorBoard sẽ hiển thị loss và accuracy theo thời gian.

In [ ]:
#docs_infra: no_execute
%tensorboard --logdir logs

<!-- <img class="tfo-display-only-on-site" src="images/word2vec_tensorboard.png"/> -->

## Tra Cứu và Phân Tích Embedding

### Trích Xuất Embedding và Xuất Sang TensorFlow Embedding Projector

Lấy các trọng số (weights) từ mô hình bằng cách sử dụng `Model.get_layer` và `Layer.get_weights`. Hàm `TextVectorization.get_vocabulary` cung cấp từ vựng để xây dựng một file metadata với một token trên mỗi dòng.

Sau khi huấn luyện:

1. Lấy trọng số embedding từ lớp `w2v_embedding`
2. Lấy từ vựng từ `vectorize_layer`

In [ ]:
weights = word2vec.get_layer('w2v_embedding').get_weights()[0]
vocab = vectorize_layer.get_vocabulary()

Tạo và lưu các file vectors và metadata:

Ghi ra hai file:
- `vectors.tsv`: Mỗi dòng là một vector embedding (`embedding_dim` giá trị, cách nhau bằng tab).
- `metadata.tsv`: Mỗi dòng là một token tương ứng với dòng trong `vectors.tsv`.

In [ ]:
out_v = io.open('vectors.tsv', 'w', encoding='utf-8')
out_m = io.open('metadata.tsv', 'w', encoding='utf-8')

for index, word in enumerate(vocab):
  if index == 0:
    continue  # skip 0, it's padding.
  vec = weights[index]
  out_v.write('\t'.join([str(x) for x in vec]) + "\n")
  out_m.write(word + "\n")
out_v.close()
out_m.close()

Tải xuống `vectors.tsv` và `metadata.tsv` để phân tích các embeddings thu được trong [Embedding Projector](https://projector.tensorflow.org/):

Tải hai file `vectors.tsv` và `metadata.tsv` lên:
- **https://projector.tensorflow.org/**
- Chọn phương pháp giảm chiều (PCA, t-SNE, UMAP) để trực quan hóa.
- Có thể:
  - Tìm kiếm theo từ khóa.
  - Xem các từ gần nhau trong không gian embedding.
  - Khám phá cấu trúc ngữ nghĩa được mô hình học được.

In [ ]:
try:
  from google.colab import files
  files.download('vectors.tsv')
  files.download('metadata.tsv')
except Exception:
  pass

## Các Bước Tiếp Theo

Tutorial này đã chỉ cho bạn cách triển khai một mô hình word2vec skip-gram với negative sampling từ đầu và trực quan hóa các word embeddings thu được.

### Tóm Tắt Toàn Bộ Notebook

Notebook này minh họa toàn bộ pipeline word2vec (skip-gram + negative sampling) với TensorFlow:

1. **Tiền xử lý dữ liệu**:
   - Tokenize, vectorize, tạo từ vựng với `TextVectorization`.

2. **Sinh dữ liệu huấn luyện**:
   - Tạo cặp skip-gram positive bằng `skipgrams`.
   - Dùng `log_uniform_candidate_sampler` để sinh negative samples.
   - Xây dựng `targets`, `contexts`, `labels`.

3. **Tối ưu pipeline**:
   - Dùng `tf.data.Dataset` với `shuffle`, `batch`, `cache`, `prefetch`.

4. **Xây dựng mô hình**:
   - Keras Subclassing `Word2Vec` với hai lớp `Embedding` và phép dot-product.

5. **Huấn luyện**:
   - Dùng loss dạng Categorical Cross Entropy/negative sampling.
   - Theo dõi bằng TensorBoard.

6. **Phân tích embedding**:
   - Xuất embedding sang `vectors.tsv` + `metadata.tsv`.
   - Trực quan hóa bằng TensorFlow Embedding Projector.

### Tài Nguyên Để Học Thêm

* Để tìm hiểu thêm về word vectors và các biểu diễn toán học của chúng, tham khảo các [notes](https://web.stanford.edu/class/cs224n/readings/cs224n-2019-notes01-wordvecs1.pdf).

* Để tìm hiểu thêm về xử lý văn bản nâng cao, đọc tutorial [Transformer model for language understanding](https://www.tensorflow.org/tutorials/text/transformer).

* Nếu bạn quan tâm đến các mô hình embedding đã được huấn luyện trước, bạn cũng có thể quan tâm đến [Exploring the TF-Hub CORD-19 Swivel Embeddings](https://www.tensorflow.org/hub/tutorials/cord_19_embeddings_keras), hoặc [Multilingual Universal Sentence Encoder](https://www.tensorflow.org/hub/tutorials/cross_lingual_similarity_with_tf_hub_multilingual_universal_encoder).

* Bạn cũng có thể muốn huấn luyện mô hình trên một tập dữ liệu mới (có nhiều tập dữ liệu có sẵn trong [TensorFlow Datasets](https://www.tensorflow.org/datasets)).

### Ứng Dụng Thực Tế

Bạn có thể sử dụng notebook này để:

- Huấn luyện embedding trên tập dữ liệu của riêng mình (tiếng Việt, domain đặc thù…).
- Điều chỉnh hyper-parameters (kích thước vector, cửa sổ ngữ cảnh, số negative samples…).
- Tái sử dụng embedding để giải các bài toán khác như:
  - Phân loại văn bản
  - Tìm kiếm văn bản/tài liệu tương tự
  - Recommendation dựa trên ngữ nghĩa
  - Và nhiều ứng dụng NLP khác